In [1]:
# import native python packages
import glob
from fnmatch import fnmatch
import multiprocessing as mp
import os
from datetime import datetime

# import custom codes
import sima_motion_bidi_correction 
import calculate_neuropil
import single_file_process

import sima
import sima.motion
import numpy as np
import os
import pickle
import h5py
import sys
from sima import sequence
import time
import bidi_offset_correction
from contextlib import contextmanager
import matplotlib
import matplotlib.pyplot as plt
import tifffile as tiff
import utils

In [2]:

def define_fparams():
    fparams = [

        # ONLY EDIT LINES BELOW THIS COMMENT

        {
            'fname': '59879-0 666um 200pwr 10xobj 4frm avg suc70 quin30-000.tiff',
            'fdir': r'D:\bruker_data\Sean\59879-0 666um 200pwr 10xobj 4frm avg suc70 quin30-000\max_disp_tests',
            'motion_correct': True,
            'signal_extract': False,
            'npil_correct': False,

            # optional args (can delete and will set to default values
            'max_disp': [[2, 2], [5, 5], [10, 10], [20, 20], [30, 50],[70, 70], [100,100] ],
            'neuropil_radius': 50,
            'min_neuropil_radius': 15,
            'flag_bidi_corr': True,
            'save_displacement': False,
            'fs': 5
        }

        # ONLY EDIT LINES ABOVE THIS COMMENT

    ]

    return fparams


def save_projections(save_dir, data_in):

    if not os.path.exists(save_dir):
        os.mkdir(save_dir)

    max_img = utils.uint8_arr(np.max(data_in, axis=0))
    mean_img = utils.uint8_arr(np.mean(data_in, axis=0))
    std_img = utils.uint8_arr(np.std(data_in, axis=0))

    tiff.imwrite(os.path.join(save_dir, 'mean_img.tif'), mean_img)
    tiff.imwrite(os.path.join(save_dir, 'max_img.tif'), max_img)
    tiff.imwrite(os.path.join(save_dir, 'std_img.tif'), std_img)

In [3]:
fparams = define_fparams()

In [4]:
fparam = fparams[0]

for max_disp in fparam['max_disp']:
    
    print('{}_{}'.format(max_disp[0], max_disp[1]))
    
    fpath = os.path.join(fparam['fdir'], fparam['fname'])
    fdir  = fparam['fdir']
    fname = os.path.splitext(fparam['fname'])[0]
    fext  = os.path.splitext(fparam['fname'])[1]
    
    save_dir = os.path.join(fparam['fdir'], '{}_{}'.format(max_disp[0], max_disp[1]))
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
        

    if fext == '.tif' or fext == '.tiff':
        # sequence: object that contains record of whole dataset; data not stored into memory all at once
        sequences = [sima.Sequence.create('TIFF', fpath)]
    elif fext == '.h5':
        sequences = [sima.Sequence.create('HDF5', fpath, 'tyx')]
    else:
        raise Exception('Inappropriate file extension')

    if not os.path.exists(os.path.join(save_dir, fname + '_mc.sima')):
        start_time = time.time()
        # define motion correction method
        # n_processes can only handle =1! Bug in their code where >1 runs into an error
        # max_displacement: The maximum allowed displacement magnitudes in pixels in [y,x]
        mc_approach = sima.motion.HiddenMarkov2D(granularity='row', max_displacement=max_disp, n_processes=1, verbose=True)

        # apply motion correction to data
        dataset = mc_approach.correct(sequences, os.path.join(save_dir, fname + '_mc.sima'),
                                      channel_names=['GCaMP'])
        # dataset dimensions are frame, plane, row(y), column (x), channel

        # use sima's fill_gaps function to interpolate missing data from motion correction
        # dtype can be changed to int16 since none of values are floats
        data_mc = np.empty(dataset[0]._sequences[0].shape, dtype='int16')
        filled_data = sequence._fill_gaps(iter(dataset[0]._sequences[0]), iter(dataset[0]._sequences[0]))
        for f_idx, frame in enumerate(filled_data):
            data_mc[f_idx, ...] = frame
        filled_data = None # clear filled_data intermediate variable
        data_mc = np.squeeze(data_mc)
        
        my_bidi_corr_obj = bidi_offset_correction.bidi_offset_correction(data_mc)  # initialize data to object
        my_bidi_corr_obj.compute_mean_image()  # compute mean image across time
        my_bidi_corr_obj.determine_bidi_offset()  # calculated bidirectional offset via fft cross-correlation
        data_out, bidi_offset = my_bidi_corr_obj.correct_bidi_frames()  # apply bidi offset to data

        save_projections(save_dir, data_out)
        
        # save motion-corrected, bidi offset corrected dataset
        sima_mc_bidi_outpath = os.path.join(save_dir, fname + '_sima_mc.h5')
        h5_write_bidi_corr = h5py.File(sima_mc_bidi_outpath, 'w')
        h5_write_bidi_corr.create_dataset('imaging', data=data_out)
        h5_write_bidi_corr.close()
        
        

2_2
Estimating model parameters.
Estimating displacements for cycle  0
Calculated bidirectional offset: 0
5_5
Estimating model parameters.
Estimating displacements for cycle  0
Calculated bidirectional offset: 0
10_10
Estimating model parameters.
Estimating displacements for cycle  0
Calculated bidirectional offset: 0
20_20
Estimating model parameters.
Estimating displacements for cycle  0
Calculated bidirectional offset: 0
30_50
Estimating model parameters.
Estimating displacements for cycle  0
Calculated bidirectional offset: 0
70_70
Estimating model parameters.
Estimating displacements for cycle  0
Calculated bidirectional offset: 0
100_100
Estimating model parameters.
Estimating displacements for cycle  0
Calculated bidirectional offset: 0
